In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score
import os
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Add /opt/homebrew/bin to the PATH
os.environ['PATH'] += ':/opt/homebrew/bin'
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-10-31 22:35:34--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-10-31 22:35:34--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv.3’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.04s   

2024-10-31 22:35:34 (9.10 MB/s) - ‘jamb_exam_resu

In [3]:
df=pd.read_csv('jamb_exam_results.csv')

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
del df['student_id']

In [6]:
df.fillna(0, inplace=True)

In [7]:
df

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,0,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,183,20,74,2,10.6,Public,Urban,Yes,No,Low,Low,16,Male,Medium,Primary,2
4996,179,0,80,2,20.0,Public,Rural,No,Yes,Medium,Medium,22,Male,Low,Secondary,1
4997,261,17,89,3,11.3,Public,Urban,No,No,Low,High,18,Male,Medium,Primary,3
4998,183,15,96,2,15.9,Public,Rural,No,No,Low,Medium,18,Male,Medium,Secondary,1


In [8]:
df.isna().sum()

jamb_score                      0
study_hours_per_week            0
attendance_rate                 0
teacher_quality                 0
distance_to_school              0
school_type                     0
school_location                 0
extra_tutorials                 0
access_to_learning_materials    0
parent_involvement              0
it_knowledge                    0
age                             0
gender                          0
socioeconomic_status            0
parent_education_level          0
assignments_completed           0
dtype: int64

## Question 1

In [18]:
# Split the data into training, validation, and test sets (60/20/20)
train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=1)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=1)


X_train_dict = train_df.drop('jamb_score', axis=1).to_dict(orient='records')
X_val_dict = val_df.drop('jamb_score', axis=1).to_dict(orient='records')
X_test_dict = test_df.drop('jamb_score', axis=1).to_dict(orient='records')


vec = DictVectorizer(sparse=True)
vec.fit(X_train_dict)

X_train = vec.transform(X_train_dict)
X_val = vec.transform(X_val_dict)
X_test = vec.transform(X_test_dict)

# # Extract target variable (jamb_score) from each dataset
y_train = train_df['jamb_score']
y_val = val_df['jamb_score']
y_test = test_df['jamb_score']

# # Train a Decision Tree Regressor with max_depth=1
regressor = DecisionTreeRegressor(max_depth=1)
regressor.fit(X_train, y_train)

# # Print the feature importances
feature_names = vec.get_feature_names_out()
print("\nFeature Importances:", dict(zip(feature_names, regressor.feature_importances_)))

# # Find the feature used for the first split
most_important_feature_index = regressor.tree_.feature[0]
most_important_feature = feature_names[most_important_feature_index]
print("\nFeature used for splitting the data:", most_important_feature)


Feature Importances: {'access_to_learning_materials=No': 0.0, 'access_to_learning_materials=Yes': 0.0, 'age': 0.0, 'assignments_completed': 0.0, 'attendance_rate': 0.0, 'distance_to_school': 0.0, 'extra_tutorials=No': 0.0, 'extra_tutorials=Yes': 0.0, 'gender=Female': 0.0, 'gender=Male': 0.0, 'it_knowledge=High': 0.0, 'it_knowledge=Low': 0.0, 'it_knowledge=Medium': 0.0, 'parent_education_level': 0.0, 'parent_education_level=Primary': 0.0, 'parent_education_level=Secondary': 0.0, 'parent_education_level=Tertiary': 0.0, 'parent_involvement=High': 0.0, 'parent_involvement=Low': 0.0, 'parent_involvement=Medium': 0.0, 'school_location=Rural': 0.0, 'school_location=Urban': 0.0, 'school_type=Private': 0.0, 'school_type=Public': 0.0, 'socioeconomic_status=High': 0.0, 'socioeconomic_status=Low': 0.0, 'socioeconomic_status=Medium': 0.0, 'study_hours_per_week': 1.0, 'teacher_quality': 0.0}

Feature used for splitting the data: study_hours_per_week


## Question 2

In [20]:
# # Split the data into training, validation, and test sets (60/20/20)
# train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=1)
# train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=1)

# # Convert the DataFrames to lists of dictionaries
# train_dict = train_df.to_dict(orient='records')
# val_dict = val_df.to_dict(orient='records')
# test_dict = test_df.to_dict(orient='records')

# # Initialize and fit the DictVectorizer
# vec = DictVectorizer(sparse=True)
# vec.fit(train_dict)

# # Transform the training, validation, and test data
# X_train = vec.transform(train_dict)
# X_val = vec.transform(val_dict)
# X_test = vec.transform(test_dict)

# # Extract target variable (jamb_score) from each dataset
# y_train = train_df['jamb_score']
# y_val = val_df['jamb_score']
# y_test = test_df['jamb_score']

# Train a Random Forest Regressor with specified parameters
rf_regressor = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_regressor.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = rf_regressor.predict(X_val)

# Calculate RMSE for the validation set
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print("RMSE on validation set:", rmse)

RMSE on validation set: 42.13724207871227


## Question 3

In [21]:
# Iterate over n_estimators from 10 to 200 with step 10 and record the RMSE
rmse_scores = []
n_estimators_range = range(10, 201, 10)

for n in n_estimators_range:
    # Train the Random Forest Regressor
    rf_regressor = RandomForestRegressor(n_estimators=n, random_state=1)
    rf_regressor.fit(X_train, y_train)
    
    # Predict on the validation set
    y_val_pred = rf_regressor.predict(X_val)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    rmse_scores.append((n, round(rmse, 3)))

# Print the RMSE scores
for n, rmse in rmse_scores:
    print(f"n_estimators={n}, RMSE={rmse}")

# Find after which value the RMSE stops improving significantly
min_rmse = min(rmse_scores, key=lambda x: x[1])
print("\nMinimum RMSE found:")
print(f"n_estimators={min_rmse[0]}, RMSE={min_rmse[1]}")

n_estimators=10, RMSE=42.137
n_estimators=20, RMSE=41.461
n_estimators=30, RMSE=41.106
n_estimators=40, RMSE=40.917
n_estimators=50, RMSE=40.852
n_estimators=60, RMSE=40.784
n_estimators=70, RMSE=40.677
n_estimators=80, RMSE=40.539
n_estimators=90, RMSE=40.504
n_estimators=100, RMSE=40.517
n_estimators=110, RMSE=40.593
n_estimators=120, RMSE=40.625
n_estimators=130, RMSE=40.651
n_estimators=140, RMSE=40.595
n_estimators=150, RMSE=40.597
n_estimators=160, RMSE=40.604
n_estimators=170, RMSE=40.628
n_estimators=180, RMSE=40.641
n_estimators=190, RMSE=40.631
n_estimators=200, RMSE=40.601

Minimum RMSE found:
n_estimators=90, RMSE=40.504


## Question 4

In [23]:
# Parameters to test
max_depth_values = [10, 15, 20, 25]
n_estimators_range = range(10, 201, 10)

# Dictionary to store mean RMSE for each max_depth
mean_rmse_results = {}

for max_depth in max_depth_values:
    rmse_list = []
    for n in n_estimators_range:
        # Train the Random Forest Regressor with given max_depth and n_estimators
        rf_regressor = RandomForestRegressor(n_estimators=n, max_depth=max_depth, random_state=1)
        rf_regressor.fit(X_train, y_train)
        
        # Predict on the validation set
        y_val_pred = rf_regressor.predict(X_val)
        
        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        rmse_list.append(rmse)
    
    # Calculate the mean RMSE for the current max_depth
    mean_rmse = np.mean(rmse_list)
    mean_rmse_results[max_depth] = round(mean_rmse, 3)

# Print the mean RMSE for each max_depth
for max_depth, mean_rmse in mean_rmse_results.items():
    print(f"max_depth={max_depth}, Mean RMSE={mean_rmse}")

# Find the best max_depth based on the minimum mean RMSE
best_max_depth = min(mean_rmse_results, key=mean_rmse_results.get)
print("\nBest max_depth based on mean RMSE:")
print(f"max_depth={best_max_depth}, Mean RMSE={mean_rmse_results[best_max_depth]}")

max_depth=10, Mean RMSE=40.392
max_depth=15, Mean RMSE=40.735
max_depth=20, Mean RMSE=40.74
max_depth=25, Mean RMSE=40.788

Best max_depth based on mean RMSE:
max_depth=10, Mean RMSE=40.392


## Question 5

In [24]:
# Train the Random Forest Regressor with specified parameters
rf_regressor = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_regressor.fit(X_train, y_train)

# Get the feature importances
feature_importances = rf_regressor.feature_importances_
feature_names = vec.get_feature_names_out()

# Print the feature importances
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importances:")
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.3f}")

# Print the most important feature
most_important_feature = sorted_feature_importance[0][0]
print("\nMost important feature:", most_important_feature)

Feature Importances:
study_hours_per_week: 0.248
attendance_rate: 0.150
distance_to_school: 0.136
teacher_quality: 0.083
age: 0.069
assignments_completed: 0.032
socioeconomic_status=High: 0.026
parent_involvement=High: 0.023
it_knowledge=High: 0.018
parent_education_level=Secondary: 0.017
parent_education_level=Primary: 0.015
parent_education_level=Tertiary: 0.014
extra_tutorials=No: 0.013
parent_involvement=Low: 0.013
it_knowledge=Low: 0.012
access_to_learning_materials=No: 0.012
parent_involvement=Medium: 0.011
socioeconomic_status=Low: 0.011
socioeconomic_status=Medium: 0.011
gender=Male: 0.010
access_to_learning_materials=Yes: 0.010
school_location=Rural: 0.010
gender=Female: 0.009
school_location=Urban: 0.009
it_knowledge=Medium: 0.009
extra_tutorials=Yes: 0.009
school_type=Private: 0.009
school_type=Public: 0.008
parent_education_level: 0.000

Most important feature: study_hours_per_week


## Question 6

In [25]:
# Split the data into training, validation, and test sets (60/20/20)
train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=1)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=1)

X_train_dict = train_df.drop('jamb_score', axis=1).to_dict(orient='records')
X_val_dict = val_df.drop('jamb_score', axis=1).to_dict(orient='records')
X_test_dict = test_df.drop('jamb_score', axis=1).to_dict(orient='records')


vec = DictVectorizer(sparse=False)
vec.fit(X_train_dict)

X_train = vec.transform(X_train_dict)
X_val = vec.transform(X_val_dict)
X_test = vec.transform(X_test_dict)

# Extract target variable (jamb_score) from each dataset
y_train = train_df['jamb_score'].values
y_val = val_df['jamb_score'].values

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Create a watchlist for tracking performance on train and validation sets
watchlist = [(dtrain, 'train'), (dval, 'eval')]

# Train the model with eta=0.3
xgb_params_eta_03 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model_eta_03 = xgb.train(xgb_params_eta_03, dtrain, num_boost_round=100, evals=watchlist, verbose_eval=False)
y_val_pred_eta_03 = model_eta_03.predict(dval)
rmse_eta_03 = np.sqrt(mean_squared_error(y_val, y_val_pred_eta_03))
print(f"RMSE with eta=0.3: {rmse_eta_03:.3f}")

# Train the model with eta=0.1
xgb_params_eta_01 = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model_eta_01 = xgb.train(xgb_params_eta_01, dtrain, num_boost_round=100, evals=watchlist, verbose_eval=False)
y_val_pred_eta_01 = model_eta_01.predict(dval)
rmse_eta_01 = np.sqrt(mean_squared_error(y_val, y_val_pred_eta_01))
print(f"RMSE with eta=0.1: {rmse_eta_01:.3f}")

# Compare results
if rmse_eta_03 < rmse_eta_01:
    print("\nThe best RMSE is achieved with eta=0.3.")
elif rmse_eta_01 < rmse_eta_03:
    print("\nThe best RMSE is achieved with eta=0.1.")
else:
    print("\nBoth eta=0.3 and eta=0.1 give the same RMSE.")

RMSE with eta=0.3: 44.432
RMSE with eta=0.1: 41.043

The best RMSE is achieved with eta=0.1.
